## American Community Survey 2016 (Millennials and Ethnicity by ZIP)

In [1]:
import os
import pandas as pd
import numpy as np
import requests
import copy
from uszipcode import ZipcodeSearchEngine
from census import Census
from pprint import pprint

# American Community Survey API Key
from config import acs_key

In [2]:
acs5_api_root = "https://api.census.gov/data/2016/acs/acs5?"
acs5_variables = "B19013_001E,\
B01003_001E,\
B01002_001E,\
B01001_002E,\
B01001_010E,\
B01001_011E,\
B01001_012E,\
B01001_026E,\
B01001_034E,\
B01001_035E,\
B01001_036E,\
B02001_001E,\
B02001_002E,\
B02001_003E,\
B02001_004E,\
B02001_005E,\
B02001_006E"

#Query examples: https://api.census.gov/data/2016/acs/acs5/examples.html

#If querying by ZCTA, use the lines below:
geo_unit = "zip%20code%20tabulation%20area:*"
predicates = f"&for={geo_unit}"

#If querying by block group, use the lines below (ignore - using ZIP)
#geo_unit = "block%20group:*"
#state_id = "17"
#county_id = "031"
#tract_id = "*"
#predicates = f"&for={geo_unit}\
#&in=state:{state_id}\
#&in=county:{county_id}\
#&in=tract:{tract_id}"


request_url = f"{acs5_api_root}\
get=NAME,{acs5_variables}\
{predicates}\
&key={acs_key}"

print(request_url)

https://api.census.gov/data/2016/acs/acs5?get=NAME,B19013_001E,B01003_001E,B01002_001E,B01001_002E,B01001_010E,B01001_011E,B01001_012E,B01001_026E,B01001_034E,B01001_035E,B01001_036E,B02001_001E,B02001_002E,B02001_003E,B02001_004E,B02001_005E,B02001_006E&for=zip%20code%20tabulation%20area:*&key=86ed69bb313729e12eb61b2d3445f49038b85726


In [3]:
acs5_response = requests.get(request_url)
acs5_json = acs5_response.json()

In [4]:
# Convert to DataFrame
acs5_header = acs5_json[0]
acs5_df = pd.DataFrame(acs5_json[1:], columns=acs5_header)

# Column Reordering
acs5_df = acs5_df.rename(columns={"B19013_001E": "Median HH Income (12 mo)",
                                   "B01003_001E": "Total Population",
                                   "B01002_001E": "Median Age",
                                   "B01001_002E": "Total Pop (M)",
                                   "B01001_010E": "22-24 yo (M)",
                                   "B01001_011E": "25-29 yo (M)",
                                   "B01001_012E": "30-34 yo (M)",
                                   "B01001_026E": "Total Pop (F)",
                                   "B01001_034E": "22-24 yo (F)",
                                   "B01001_035E": "25-29 yo (F)",
                                   "B01001_036E": "30-34 yo (F)",
                                   "B02001_001E": "Total Pop (Race)",
                                   "B02001_002E": "Total Pop (Race-W)",
                                   "B02001_003E": "Total Pop (Race-B)",
                                   "B02001_004E": "Total Pop (Race-N)",
                                   "B02001_005E": "Total Pop (Race-A)",
                                   "B02001_006E": "Total Pop (Race-H)"
                                })

In [5]:
for c in acs5_df.loc[:, 'Median HH Income (12 mo)':'Total Pop (Race-H)']:
    acs5_df[c] = pd.to_numeric(acs5_df[c], errors='ignore')

In [6]:
acs5_df.head()

,NAME,Median HH Income (12 mo),Total Population,Median Age,Total Pop (M),22-24 yo (M),25-29 yo (M),30-34 yo (M),Total Pop (F),22-24 yo (F),25-29 yo (F),30-34 yo (F),Total Pop (Race),Total Pop (Race-W),Total Pop (Race-B),Total Pop (Race-N),Total Pop (Race-A),Total Pop (Race-H),zip code tabulation area
0,ZCTA5 01001,56714,17423,45.0,8059,396,415,583,9364,307,554,619,17423,15974,256,5,703,24,01001
1,ZCTA5 01002,48923,29970,23.2,14536,2104,1019,568,15434,1588,878,615,29970,23459,1711,140,3502,17,01002
2,ZCTA5 01003,2499,11296,19.9,5694,472,70,1,5602,456,9,0,11296,8698,660,30,1538,0,01003
3,ZCTA5 01005,70568,5228,44.1,2798,22,142,158,2430,132,82,47,5228,5063,105,0,32,18,01005
4,ZCTA5 01007,80502,14888,42.5,7224,413,216,323,7664,192,336,327,14888,13906,125,0,443,0,01007


In [7]:
#--------------age--------------#

#combine acs5 age group columns into one millenial group column per sex

    #males who are 22-34 yo
acs5_df['22 - 34 yo (M)'] = acs5_df["22-24 yo (M)"] + acs5_df["25-29 yo (M)"] + acs5_df["30-34 yo (M)"]

    #females who are 22-34 yo
acs5_df['22 - 34 yo (F)'] = acs5_df["22-24 yo (F)"] + acs5_df["25-29 yo (F)"] + acs5_df["30-34 yo (F)"]

    #total population that is 22-34 yo
acs5_df['22 - 34 yo (Total)'] = acs5_df['22 - 34 yo (M)'] + acs5_df['22 - 34 yo (F)']

    #% males who are 22-34 yo
acs5_df['22 - 34 yo (% Pop M)'] = acs5_df['22 - 34 yo (M)']/acs5_df['Total Pop (M)']

    #% females who are 22-34 yo
acs5_df['22 - 34 yo (% Pop F)'] = acs5_df['22 - 34 yo (F)']/acs5_df['Total Pop (F)']

    #% population that is 22-34 yo
acs5_df['22 - 34 yo (% Total)'] = acs5_df['22 - 34 yo (Total)']/acs5_df['Total Population']

    #% population that is 22-34 yo
acs5_df['22 - 34 yo (% Total)'] = acs5_df['22 - 34 yo (Total)']/acs5_df['Total Population']

#--------------race--------------#

    #% population that is White
acs5_df['Race-W (% Total)'] = acs5_df['Total Pop (Race-W)']/acs5_df['Total Population']

    #% population that is Black
acs5_df['Race-B (% Total)'] = acs5_df['Total Pop (Race-B)']/acs5_df['Total Population']

    #% population that is Native American
acs5_df['Race-N (% Total)'] = acs5_df['Total Pop (Race-N)']/acs5_df['Total Population']

    #% population that is Asian
acs5_df['Race-A (% Total)'] = acs5_df['Total Pop (Race-A)']/acs5_df['Total Population']

    #% population that is Native Hawaiian/Pacific Islander 
acs5_df['Race-H (% Total)'] = acs5_df['Total Pop (Race-H)']/acs5_df['Total Population']

acs5_df.head()

,NAME,Median HH Income (12 mo),Total Population,Median Age,Total Pop (M),22-24 yo (M),25-29 yo (M),30-34 yo (M),Total Pop (F),22-24 yo (F),...,22 - 34 yo (F),22 - 34 yo (Total),22 - 34 yo (% Pop M),22 - 34 yo (% Pop F),22 - 34 yo (% Total),Race-W (% Total),Race-B (% Total),Race-N (% Total),Race-A (% Total),Race-H (% Total)
0,ZCTA5 01001,56714,17423,45.0,8059,396,415,583,9364,307,...,1480,2874,0.172974,0.158052,0.164954,0.916834,0.014693,0.000287,0.040349,0.001377
1,ZCTA5 01002,48923,29970,23.2,14536,2104,1019,568,15434,1588,...,3081,6772,0.253921,0.199624,0.225959,0.782749,0.057090,0.004671,0.116850,0.000567
2,ZCTA5 01003,2499,11296,19.9,5694,472,70,1,5602,456,...,465,1008,0.095364,0.083006,0.089235,0.770007,0.058428,0.002656,0.136154,0.000000
3,ZCTA5 01005,70568,5228,44.1,2798,22,142,158,2430,132,...,261,583,0.115082,0.107407,0.111515,0.968439,0.020084,0.000000,0.006121,0.003443
4,ZCTA5 01007,80502,14888,42.5,7224,413,216,323,7664,192,...,855,1807,0.131783,0.111561,0.121373,0.934041,0.008396,0.000000,0.029756,0.000000


In [8]:
#trim down what's in the ACS5 data since some columns are just used for calculations
acs5_df_2 = acs5_df.loc[:,['zip code tabulation area', 'Median HH Income (12 mo)', 'Total Population', 'Median Age', '22 - 34 yo (% Pop M)','22 - 34 yo (% Pop F)','22 - 34 yo (% Total)','Race-W (% Total)', 'Race-B (% Total)', 'Race-N (% Total)', 'Race-A (% Total)', 'Race-H (% Total)']]
acs5_df_2 = acs5_df_2.rename(columns={'zip code tabulation area': 'ZCTA (~ZIP)'})
acs5_df_2.head()

,ZCTA (~ZIP),Median HH Income (12 mo),Total Population,Median Age,22 - 34 yo (% Pop M),22 - 34 yo (% Pop F),22 - 34 yo (% Total),Race-W (% Total),Race-B (% Total),Race-N (% Total),Race-A (% Total),Race-H (% Total)
0,01001,56714,17423,45.0,0.172974,0.158052,0.164954,0.916834,0.014693,0.000287,0.040349,0.001377
1,01002,48923,29970,23.2,0.253921,0.199624,0.225959,0.782749,0.057090,0.004671,0.116850,0.000567
2,01003,2499,11296,19.9,0.095364,0.083006,0.089235,0.770007,0.058428,0.002656,0.136154,0.000000
3,01005,70568,5228,44.1,0.115082,0.107407,0.111515,0.968439,0.020084,0.000000,0.006121,0.003443
4,01007,80502,14888,42.5,0.131783,0.111561,0.121373,0.934041,0.008396,0.000000,0.029756,0.000000


In [9]:
#*-----saving off csv copy of aggregated ACS data first-----*
acs5_df_2.to_csv("Resources\\ACS_2016_census_pop_char_by_zip.csv",index=False)